In [1]:
import numpy as np
from collections import defaultdict
import random
from utils import emotion_scores
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [2]:
class BigramLM:
    def __init__(self):
        self.vocab = set()
        self.bigram_counts = defaultdict(lambda: defaultdict(int))
        self.unigram_counts = defaultdict(int)
        self.bigram_probs = None
        self.beta_values = None
        self.emotion_dict = {'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}
        self.sentence_emotions = []
        self.bigram_sentences = defaultdict(lambda: defaultdict(list))

    def learn_from_dataset(self, dataset):
        for x, sentence in enumerate(dataset):
            emotion = emotion_scores(sentence)
            self.sentence_emotions.append(emotion)
            tokens = sentence.split()                        
            for i in range(len(tokens) - 1):
                word1, word2 = tokens[i], tokens[i + 1]
                self.vocab.add(word1)
                self.vocab.add(word2)
                self.bigram_counts[word1][word2] += 1
                self.bigram_sentences[word1][word2].append(x)
                self.unigram_counts[word1] += 1                            
                
        self.vocab = list(self.vocab)
        print(f"Vocabulary size: {len(self.vocab)}")

    def calculate_beta_values_sentence(self):  
        num_words = len(self.vocab)
        self.beta_values_sentence = np.zeros((num_words, num_words, 6))

        for i, word1 in tqdm(enumerate(self.vocab)):
            if word1 not in self.bigram_counts.keys():
                continue
            for j, word2 in enumerate(self.vocab):
                if word2 not in self.bigram_counts[word1].keys():
                    continue
                self.beta_values_sentence[i][j] = np.array([np.mean([self.sentence_emotions[sentence][k]['score'] for sentence in self.bigram_sentences[word1][word2]]) for k in range(6)])
                

    def calculate_beta_values(self):
            num_words = len(self.vocab)
            self.beta_values = np.zeros((num_words, num_words, 6))
            for i, word1 in tqdm(enumerate(self.vocab)):
                if word1 not in self.bigram_counts.keys():
                    continue
                for j, word2 in enumerate(self.vocab):
                    if word2 not in self.bigram_counts[word1].keys():
                        continue
                    emotions = emotion_scores(word1 + " " + word2)
                    self.beta_values[i][j] = np.array([emotions[k]['score'] for k in range(6)])
                    
    def calculate_bigram_probs(self):
        num_words = len(self.vocab)
        self.bigram_probs = np.zeros((num_words, num_words))        
        
        for i, word1 in tqdm(enumerate(self.vocab)):
            for j, word2 in enumerate(self.vocab):
                if self.unigram_counts[word1] > 0:
                    self.bigram_probs[i][j] = float(self.bigram_counts[word1][word2]) / float(self.unigram_counts[word1])

                    
    def calculate_bigram_probs_laplace(self):
        num_words = len(self.vocab)
        self.bigram_probs = np.zeros((num_words, num_words))

        for i, word1 in enumerate(self.vocab):
            for j, word2 in enumerate(self.vocab):
                self.bigram_probs[i][j] = (self.bigram_counts[word1][word2] + 1) / (self.unigram_counts[word1] + num_words)
                
    
    def calculate_bigram_probs_kneser_ney(self, discount=0.75):
        num_words = len(self.vocab)
        self.bigram_probs = np.zeros((num_words, num_words))

        continuation_counts = defaultdict(set)
   
        for word1, word2_dict in self.bigram_counts.items():
            for word2 in word2_dict:
                continuation_counts[word2].add(word1)

        total_continuations = {word2: len(word1s) for word2, word1s in continuation_counts.items()}

        for i, word1 in tqdm(enumerate(self.vocab)):
            sum_adjusted_counts = sum(max(self.bigram_counts[word1][word2] - discount, 0) for word2 in self.vocab)
            for j, word2 in enumerate(self.vocab):
                if word2 in total_continuations.keys():
                    adjusted_count = max(self.bigram_counts[word1][word2] - discount, 0)
                    continuation_prob = total_continuations[word2] / sum(total_continuations.values()) if sum(total_continuations.values()) > 0 else 0
                    lower_order_weight = (discount * continuation_prob) / self.unigram_counts[word1] if self.unigram_counts[word1] > 0 else 0

                    self.bigram_probs[i][j] = adjusted_count / self.unigram_counts[word1] + lower_order_weight if self.unigram_counts[word1] > 0 else 0
                    


In [3]:
corpus = open('../Dataset/corpus.txt')
dataset = []
for i in corpus.readlines():
    dataset.append('<SOS> ' + i + ' <EOS>')

In [4]:
model = BigramLM()
model.learn_from_dataset(dataset)

Vocabulary size: 5431


In [16]:
pickle.dump(model.vocab, open('Checkpoints/vocab.pkl', 'wb'))

In [5]:
model.calculate_beta_values_sentence()
pickle.dump(model.beta_values_sentence, open('Checkpoints/beta_values_sentence.pkl', 'wb'))

5431it [00:06, 890.43it/s] 


In [6]:
model.calculate_beta_values()

5431it [08:38, 10.47it/s]


In [7]:
pickle.dump(model.beta_values, open('Checkpoints/beta_values.pkl', 'wb'))

In [8]:
model.calculate_bigram_probs()
pickle.dump(model.bigram_probs, open('Checkpoints/bigram_probs.pkl', 'wb'))

5431it [00:16, 321.77it/s]


In [ ]:
model.calculate_bigram_probs_kneser_ney()
pickle.dump(model.bigram_probs, open('Checkpoints/bigram_probs_kneser.pkl', 'wb'))

In [10]:
model.calculate_bigram_probs_laplace()
pickle.dump(model.bigram_probs, open('Checkpoints/bigram_probs_laplace.pkl', 'wb'))